Application using altimetry data obtained from AVISO.
* Region of interest: Mediterranean Sea.
* Period of interest: May 2014.
* Satellite avaible: AltiKa, Cryosat, H2 and JASON-2. 

In [1]:
using divand
using NetCDF
using PyPlot
using Logging
Logging.configure(level=DEBUG);

# User inputs

## Domain of interest, resolution

In [2]:
lonmin, lonmax, latmin, latmax = -7., 40., 30., 48.
dx, dy, dt = .125, .125, 1.
lonr = lonmin:dx:lonmax;
latr = latmin:dy:latmax;
timer = 23520:dt:23521;

## File and directories

In [3]:
datadir = joinpath("../data/");
datafile = joinpath(datadir, "ADT_20140515_20140604.dat")
resultdir = joinpath("../results/divand/");
resultfile = joinpath(resultdir, "ADT_20140515_20140604_0pt125d1.nc");
info("Data file:\n", datafile)
info("Result file:\n", resultfile)

19-juil. 09:24:31:INFO:root:Data file:
../data/ADT_20140515_20140604.dat
19-juil. 09:24:31:INFO:root:Result file:
../results/divand/ADT_20140515_20140604_0pt125d1.nc


# Read data file (4-column, ascii)

In [4]:
function readAltimetryFile(fname)

    data = readlines(open(fname,"r"))
    nobs = length(data)

    lon = zeros(nobs)
    lat = zeros(nobs)
    time = zeros(nobs)
    field = zeros(nobs)

    for i in 1:nobs
        rec = split(data[i])
        lon[i] = parse(Float64,rec[1])
        lat[i] = parse(Float64,rec[2])
        time[i] = parse(Float64,rec[3])
        field[i] = parse(Float64,rec[4])
    end
    
    return lon, lat, time, field
end
info("Data read from file ", datafile)

19-juil. 09:24:31:INFO:root:Data read from file ../data/ADT_20140515_20140604.dat


In [5]:
lon, lat, time, field = readAltimetryFile(datafile);
info("Reading ", length(lon), " data points")
info("Mean field value = ", mean(field))

19-juil. 09:24:31:INFO:root:Reading 11371 data points
19-juil. 09:24:31:INFO:root:Mean field value = -0.09550065957259697


Create mask and metrics using coordinate vectors

In [6]:
mask, (px, py, pt), (xi, yi, ti) = divand_rectdom(lonr, latr, timer);

In [7]:
size(mask)

(377, 145, 2)

## Analysis parameters

In [8]:
# correlation length
len = (1., 1., 4.);
# obs. error variance normalized by the background error variance
epsilon2 = 1.;

# Perform interpolation

In [9]:
@time fi,s = divandrun(mask, (px, py, pt), (xi, yi, ti), (lon, lat, time), field, len, epsilon2; alphabc=2);

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] max(::Array{Float64,2}, ::Float64) at ./deprecated.jl:57
 [3] divand_bc_stretch(::BitArray{3}, ::Tuple{Array{Float64,3},Array{Float64,3},Array{Float64,3}}, ::Tuple{Array{Float64,3},Array{Float64,3},Array{Float64,3}}, ::Tuple{Float64,Float64,Float64}, ::Array{Any,1}, ::Int64) at /home/ctroupin/.julia/v0.6/divand/src/divand_bc_stretch.jl:60
 [4] #divandrun#551(::Tuple{}, ::Array{Any,1}, ::Array{Float64,1}, ::Bool, ::Bool, ::Float64, ::Int64, ::Int64, ::Tuple{}, ::Symbol, ::Array{Any,1}, ::Array{Any,1}, ::Array{Any,1}, ::Int64, ::divand.#divand_pc_none, ::divand.##553#555, ::Array{Float64,3}, ::Array{Float64,1}, ::Type{T} where T, ::Int64, ::Bool, ::Array{Any,1}, ::divand.#divandrun, ::BitArray{3}, ::Tuple{Array{Float64,3},Array{Float64,3},Array{Float64,3}}, ::Tuple{Array{Float64,3},Array{Float64,3},Array{Float64,3}}, ::Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1}}, ::Array{Float64,1}, ::Tuple{Float64,Floa

LoadError: [91mMethodError: no method matching divand.divand_constrain(::Array{Float64,1}, ::Diagonal{Int64}, ::SparseMatrixCSC{Float64,Int64})[0m
Closest candidates are:
  divand.divand_constrain(::Array{T<:AbstractFloat,1}, [91m::AbstractArray{T<:AbstractFloat,2}[39m, ::AbstractArray{T<:AbstractFloat,2}) where T<:AbstractFloat at /home/ctroupin/.julia/v0.6/divand/src/divand.jl:13[39m

## Export results to netCDF

In [10]:
sz = size(mask)
dims = [NcDim("longitude",sz[1]), NcDim("latitude",sz[2]), NcDim("time",sz[3])];
nc = NetCDF.create(resultfile, NcVar("ADT", dims))
nc["ADT"][:,:, :] = fi
NetCDF.close(nc);
info("Written interpolated field in file:\n", resultfile)

LoadError: [91mNetCDF error code 13:
	Permission denied[39m

# Plotting

Done in [another notebook in Python](../python/plot_AVISO_divand.ipynb) due to a problem with Julia / matplotlib.